In [15]:
import requests
from requests_html import HTML,HTMLSession
import pandas as pd 

import time 
import os

In [ ]:
# Execution Flow :
#     1)scarpe_and_save_csv()
#     2)scarpe_tag()
#     3)extract_data_from_url()
#     4)parse_tagged_page()
#     5)clean_scarped_data()

In [26]:
os.makedirs(os.path.join(os.getcwd(),"OUTPUT_CSVs"),exist_ok=True)
output_dir = os.path.join(os.getcwd(),"OUTPUT_CSVs")

In [2]:
# cleaning the data further
def clean_scarped_data(text,keyname=None):
    if keyname == "votes":
        return text.replace("\nvotes","") 
    if keyname == "tags":
        return list(text.split(" "))
    return text

In [3]:
# try to grab the necessary class and scrap those vital data and returned in the form of a lsit 
def parse_tagged_page(html):
    list_of_question_summary = html.find(".question-summary")
    key_names = ["question","votes","tags"]
    classes_needed = ['.question-hyperlink','.vote',".tags"]
    datas = []
    for q_el in list_of_question_summary:
        question_data = {}
        for i,lookup_class in enumerate(classes_needed):
            sub_el = q_el.find(lookup_class,first=True)
            keyname = key_names[i]
            question_data[keyname] = clean_scarped_data(sub_el.text,keyname=keyname)
        datas.append(question_data)
    return datas 

In [4]:
# makes a get request and passes the html text for parsing 
def extract_data_from_url(url):
    r = requests.get(url)
    if r.status_code not in range(200,299):
        return []
    html_str = r.text
    html = HTML(html=html_str)
    datas = parse_tagged_page(html)
    return datas

In [28]:
# we define the final endpoints with all the queries passed
def scarpe_tag(tag="python",query_filter="votes",max_pages=10,pagesize=10):
    base_url = "https://stackoverflow.com/questions/tagged/"
    datas = []
    for p in range(max_pages):
        page_num= p+1
        final_endpoint = f"{base_url}{tag}?tab={query_filter}&page={page_num}&pagesize={pagesize}"
        datas += extract_data_from_url(final_endpoint)
        time.sleep(0.5)
    return datas

In [33]:
# takes list of tags and scrapes those tags 
def scarpe_and_save_csv(tags_list=[]):
    if len(tags_list) == 0:
        raise Exception("tags_list cannot be empty")
    for tag in tags_list:
            datas = scarpe_tag(tag)
            df = pd.DataFrame(datas)
            df.head(n=10)
            df.to_csv(os.path.join(output_dir,f"{tag}.csv"),index=False)
            print(f"{tag}.csv saved")
            time.sleep(1)
    print("Scrapping Done ! Peace ")

In [34]:
scarpe_and_save_csv(['javascript','python','java'])

javascript.csv saved
python.csv saved
java.csv saved
